# Expirements

### imports

In [30]:
import duckdb 
import pandas as pd
import pyarrow.parquet as pq
from datetime import datetime
import os
import sys
from pathlib import Path

### Duckdb Parittion

In [ ]:
cwd = Path.cwd()
processed_dir = cwd / 'data' / 'processed'
raw_dir = cwd / 'data' / 'raw'

print(f'processed_dir: {processed_dir}')
print(f'raw_dir: {raw_dir}')

# Connect to duckdb in memory
con = duckdb.connect(database=':memory:')


file_types = ['customers', 'merchants', 'transactions', 'login_attempts']
partition_statement = ""

for file_type in file_types:
    con.execute(f"""
    CREATE TABLE IF NOT EXISTS {file_type} AS
    SELECT
        *
    FROM read_parquet('{raw_dir}/{file_type}.parquet')
    """)
    if file_type == 'customers':
        partition_statement = "PARTITION_BY (AccountCreationMonth)"
    if file_type == 'merchants':
        partition_statement = "PARTITION_BY (ingestion_date)"
    if file_type == 'transactions':
        partition_statement = "PARTITION_BY (TimestampMonth)"
    if file_type == 'login_attempts':
        partition_statement = "PARTITION_BY (LoginTimestampMonth)"

    statement = f"""
        COPY {file_type} TO '{processed_dir}/{file_type}' 
        (FORMAT parquet, {partition_statement}, OVERWRITE_OR_IGNORE)
    """

    # print(statement)
    
    # Write partitions
    con.execute(f"""
        COPY {file_type} TO '{processed_dir}/{file_type}' 
        (FORMAT parquet, {partition_statement}, OVERWRITE_OR_IGNORE)
    """)